## Early Warning Customer data

ID: fbc0c2a7-9b29-4ed2-a909-3e68c26bb251
Date Pulled: 2025-01-07
Data Sources:
- Okta system logs
- Crowdstrike
- TAP
- Exchange ORG details
- Active Directory

The purpose of this portion of the notebook is to examine the distribution of devices across AD data.

In [1]:
import polars as pl
import datetime as dt
import json
from typing import Any, Dict, Iterable
import os
import re
import pathlib
import zstandard as zstd
import csv
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Decompress target files
def normalize(name: str) -> str:
    name = name.lower().strip()
    name = re.sub(r'\s+', '_', name)
    name = re.sub(r'[^a-z0-9._-]', '', name)
    return name

def unzst_directory(src_dir: pathlib.Path):
    src_dir = src_dir.resolve()
    for root, _, files in os.walk(src_dir):
        root = pathlib.Path(root)
        rel = root.relative_to(src_dir)
        target_root = src_dir / rel
        target_root.mkdir(parents=True, exist_ok=True)
        for file in files:
            if file.endswith('.zst'):
                inpath = root / file
                stem = pathlib.Path(normalize(file[:-4]))
                outpath = target_root / stem
                print(f"Decompressing {inpath} → {outpath}")
                with open(inpath, 'rb') as ifh, open(outpath, 'wb') as ofh:
                    dctx = zstd.ZstdDecompressor()
                    dctx.copy_stream(ifh, ofh)

def dict_with_most_keys(dicts: Iterable[Dict[Any, Any]]) -> Dict[Any, Any]:
    """
    Returns the dictionary from the iterable that has the most keys.
    If multiple dictionaries are tied, returns the first one encountered.
    Raises ValueError if the iterable is empty.
    """
    try:
        # `max` with key=len chooses the dict with the largest number of keys
        return max(dicts, key=len)
    except ValueError as e:
        # This will occur if dicts is empty
        raise ValueError("The input iterable must contain at least one dictionary") from e



In [3]:
unzst_directory(pathlib.Path("/Users/eric.louhi/Github/reach-data-experiments/data/earlywarning"))


Decompressing /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/users.zst → /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/users
Decompressing /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/2025-01-07.zst → /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/2025-01-07
Decompressing /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/okta_logs.zst → /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/okta_logs
Decompressing /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/azure_ad.zst → /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/azure_ad
Decompressing /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/crowdstrike_configs.zst → /Users/eric.louhi/Github/reach-data-experiments/data/earlywarning/crowdstrike_configs


In [2]:
az_ad_data: list[dict[str, Any]] = []
with open("../data/earlywarning/azure_ad", 'r') as f:
    az_ad_data = json.load(f)

In [3]:
az_users = az_ad_data["users"]
az_users[0]

{'displayName': 'On-Premises Directory Synchronization Service Account',
 'mailNickname': 'Sync_S8150CAWV_2ca7fb000046',
 'userType': 'Member',
 'accountEnabled': True,
 'passwordPolicies': 'DisablePasswordExpiration',
 'userPrincipalName': 'Sync_S8150CAWV_2ca7fb000046@earlywarningservices.onmicrosoft.com',
 'id': 'aa736530-e846-4f93-bd48-9366ce532039',
 'transitiveMemberOf': [],
 'devices': []}

In [4]:
dict_with_most_keys(az_users)

{'displayName': 'Galle, Shelly',
 'mailNickname': 'joenss',
 'givenName': 'Shelly',
 'surname': 'Galle',
 'jobTitle': 'Sr. Privacy Office Manager',
 'department': 'Privacy and Data Risk Management',
 'mail': 'Shelly.Galle@earlywarning.com',
 'proxyAddresses': ['SMTP:Shelly.Galle@earlywarning.com',
  'smtp:Shelly.Joens@earlywarning.com',
  'smtp:joenss@earlywarningservices.onmicrosoft.com',
  'X500:/o=EWS/ou=Exchange Administrative Group (FYDIBOHF23SPDLT)/cn=Recipients/cn=Joens, Shelly329',
  'smtp:Shelly.Galle@earlywarningservices.mail.onmicrosoft.com',
  'x500:/o=ExchangeLabs/ou=Exchange Administrative Group (FYDIBOHF23SPDLT)/cn=Recipients/cn=ffd50fa207f94f319c6f31c4a35a056d-Galle, Shel',
  'X500:/o=EWS/ou=External (FYDIBOHF25SPDLT)/cn=Recipients/cn=cc4c193a95be4a1791b27d8a545d1728'],
 'userType': 'Member',
 'employeeType': 'Regular',
 'accountEnabled': True,
 'companyName': 'Early Warning Services, LLC',
 'onPremisesDistinguishedName': 'CN=Galle\\, Shelly,OU=FCRA Compliance,OU=Custom

In [22]:
# Load the list of dictionaries into a Polars DataFrame
az_user_df = pl.DataFrame(az_users)

# Explode the array field to individual rows, then aggregate the counts
aggregated_counts = (
    az_user_df.with_columns(pl.col("devices").list.len().alias("device_count"))  # length of each list
    .group_by("device_count")                                       # group by that length
    .agg(pl.count("id").alias("user_count"))                     # count IDs per length
    .sort(["user_count"], descending=True)
)
filtered_az_user_df = aggregated_counts.filter(pl.col("device_count")>=1)
filtered_az_user_df.write_csv("../data/earlywarning/ad-users-devices.csv")
print(filtered_az_user_df)


shape: (33, 2)
┌──────────────┬────────────┐
│ device_count ┆ user_count │
│ ---          ┆ ---        │
│ u32          ┆ u32        │
╞══════════════╪════════════╡
│ 2            ┆ 339        │
│ 1            ┆ 304        │
│ 3            ┆ 296        │
│ 4            ┆ 205        │
│ 5            ┆ 139        │
│ …            ┆ …          │
│ 54           ┆ 1          │
│ 41           ┆ 1          │
│ 38           ┆ 1          │
│ 29           ┆ 1          │
│ 26           ┆ 1          │
└──────────────┴────────────┘


In [21]:
# hist_chart
chart = filtered_az_user_df["device_count"].plot.hist()
chart

alt.Chart(...)

In [24]:
print(aggregated_counts)

shape: (34, 2)
┌──────────────┬────────────┐
│ device_count ┆ user_count │
│ ---          ┆ ---        │
│ u32          ┆ u32        │
╞══════════════╪════════════╡
│ 0            ┆ 808        │
│ 2            ┆ 339        │
│ 1            ┆ 304        │
│ 3            ┆ 296        │
│ 4            ┆ 205        │
│ …            ┆ …          │
│ 54           ┆ 1          │
│ 41           ┆ 1          │
│ 38           ┆ 1          │
│ 29           ┆ 1          │
│ 26           ┆ 1          │
└──────────────┴────────────┘


In [41]:
az_users
# pl.col("devices").list.len().alias("device_count")
filtered_az_user_device_cnt_df = az_user_df.filter(pl.col("devices").list.len()==339),


In [42]:
filtered_az_user_device_cnt_df

(shape: (0, 21)
 ┌────────────┬────────────┬──────────┬───────────┬───┬───────────┬─────────┬───────────┬───────────┐
 │ displayNam ┆ mailNickna ┆ userType ┆ accountEn ┆ … ┆ manager@d ┆ manager ┆ otherMail ┆ identitie │
 │ e          ┆ me         ┆ ---      ┆ abled     ┆   ┆ elta      ┆ ---     ┆ s         ┆ s         │
 │ ---        ┆ ---        ┆ str      ┆ ---       ┆   ┆ ---       ┆ str     ┆ ---       ┆ ---       │
 │ str        ┆ str        ┆          ┆ bool      ┆   ┆ list[stru ┆         ┆ list[str] ┆ list[stru │
 │            ┆            ┆          ┆           ┆   ┆ ct[2]]    ┆         ┆           ┆ ct[3]]    │
 ╞════════════╪════════════╪══════════╪═══════════╪═══╪═══════════╪═════════╪═══════════╪═══════════╡
 └────────────┴────────────┴──────────┴───────────┴───┴───────────┴─────────┴───────────┴───────────┘,)

In [50]:
t_df = pl.DataFrame([x for x in az_users if len(x.get("devices",[])) > 30])

In [51]:
t_df

displayName,mailNickname,givenName,surname,jobTitle,department,mail,proxyAddresses,userType,employeeType,accountEnabled,companyName,onPremisesDistinguishedName,passwordPolicies,city,state,country,streetAddress,userPrincipalName,onPremisesSamAccountName,id,manager@delta,manager,transitiveMemberOf,devices
str,str,str,str,str,str,str,list[str],str,str,bool,str,str,str,str,str,str,str,str,str,str,list[struct[2]],str,list[str],list[struct[25]]
"""Gajulapalli, Krishna""","""gajulapk""","""Krishna""","""Gajulapalli""","""Sr. Software Engineer - Java""","""Product Engineering - Zelle""","""Krishna.Gajulapalli@earlywarni…","[""SMTP:Krishna.Gajulapalli@earlywarning.com"", ""smtp:Krishna.Gajulapalli@earlywarningservices.onmicrosoft.com"", … ""X500:/o=EWS/ou=External (FYDIBOHF25SPDLT)/cn=Recipients/cn=7764e7e4db644f5bb96e8111438f27da""]","""Member""","""Regular""",true,"""Early Warning Services, LLC""","""CN=Gajulapalli\, Krishna,OU=So…","""DisablePasswordExpiration""","""Scottsdale""","""Arizona""","""United States""","""5801 N. Pima Rd""","""Krishna.Gajulapalli@earlywarni…","""gajulapk""","""391b13e0-979f-4f0d-be8e-819589…","[{""#microsoft.graph.user"",""88ab9f8f-f5e6-4bf3-b8fc-6146dfd9fe88""}]","""88ab9f8f-f5e6-4bf3-b8fc-6146df…","[""6fcd6edc-e490-443e-b2ed-06a41b9d5f4a"", ""8bc382d1-f63d-4569-a4ec-a301b756c25c"", … ""3109d342-be42-4565-8726-193fd6c27b4e""]","[{""e86aa2d4-3495-4856-a658-9717bb54a970"",""MN93XFCCX4"",""MacOS"",""12.5.1"",""RegisteredDevice"",true,""2022-09-13T17:23:01Z"",""2022-09-13T17:23:01Z"",null,""7351d8f5-b375-4182-b2c4-2b2fcc1f2980"",null,null,null,null,null,null,null,null,null,null,null,null,""2022-09-13T10:23:01Z"",""Workplace"",2}, {""c30da7db-5aaa-48cc-a1f0-0f8e9757ad43"",""MN93XFCCX4"",""MacOS"",""12.5.0"",""RegisteredDevice"",true,""2022-08-17T15:31:55Z"",""2022-08-17T15:31:55Z"",null,""eeac05a1-a455-4b94-9dd2-0092d154809e"",null,null,null,null,null,null,null,null,null,null,null,null,""2022-08-17T08:31:55Z"",""Workplace"",2}, … {""b4012aba-8df2-442b-99ca-f34e170b798f"",""MN93XFCCX4"",""MacOS"",""12.5.1"",""RegisteredDevice"",true,""2022-09-13T13:12:48Z"",""2022-09-13T13:12:48Z"",null,""144b9e12-c334-40a4-8fb9-f01ef5c7fd32"",null,null,null,null,null,null,null,null,null,null,null,null,""2022-09-13T06:12:48Z"",""Workplace"",2}]"
"""Goodgame, Christopher""","""goodgach""","""Christopher""","""Goodgame""","""IT Asset Specialist II""","""IT Asset Management""","""Christopher.Goodgame@earlywarn…","[""SMTP:Christopher.Goodgame@earlywarning.com"", ""smtp:goodgach@earlywarningservices.onmicrosoft.com"", … ""X500:/o=EWS/ou=External (FYDIBOHF25SPDLT)/cn=Recipients/cn=ed69e648351f44bfa77786f7d90c6606""]","""Member""","""Regular""",true,"""Early Warning Services, LLC""","""CN=Goodgame\, Christopher,OU=D…","""DisablePasswordExpiration""","""Scottsdale""","""Arizona""",null,"""5801 N. Pima Rd""","""Christopher.Goodgame@earlywarn…","""goodgach""","""20827d7b-40df-4486-b02d-7ed416…","[{""#microsoft.graph.user"",""d9d2b03d-3833-42bf-af5c-2cab2c861906""}]","""d9d2b03d-3833-42bf-af5c-2cab2c…","[""eb8a69d1-205a-40b3-8b8a-656865d3aac1"", ""aaca1fb1-3dfe-481d-a0f4-c77021f95937"", … ""5eee81fb-24e8-4283-9a79-db2e756a239b""]","[{""6f32770e-5a56-48f8-93d3-8dbab8a75a52"",""iPhone-DX3FXADW0DXP"",""IPhone"",""16.5"",""RegisteredDevice"",true,""2023-05-31T18:32:07Z"",""2023-05-31T18:32:07Z"",""Corporate Device"",""4b998717-7b21-49f0-86c9-aec7ce42600f"",""Company"",""iOS Devices"",""CompanyPortalBulkEnrollment"",false,false,false,""MDM"",""Apple"",""0000000a-0000-0000-c000-000000000000"",""iPhone 12"",null,null,""2023-05-31T11:32:07Z"",""Workplace"",2}, {""4fd6c79a-f344-44fc-86ef-4848c88d1b26"",""SCTL8491"",""Windows"",""10.0.16299.0"",""RegisteredDevice"",true,""2021-01-22T16:45:48Z"",""2021-01-22T16:45:48Z"",null,""4079f86a-c65b-42db-a4ac-cb82eac13033"",null,null,null,null,null,null,null,null,null,null,null,null,""2021-01-22T08:45:48Z"",""Workplace"",2}, … {""e7157191-c4b9-411f-9ec3-785af1ad8cb7"",""C02T22L5GTDY"",""MacOS"",""11.6.2"","

In [53]:
total_device_cnt = sum([len(x) for x in az_users if len(x.get("devices",[])) > 0])
total_device_cnt

36979